In [6]:
import torch
major_version,minor_version=torch.cuda.get_device_capability()
print(major_version,minor_version)

8 9


In [4]:
%%capture
# ColabÏóêÏÑú torch 2.2.1ÏùÑ ÏÇ¨Ïö©ÌïòÍ≥† ÏûàÏúºÎØÄÎ°ú, Ìå®ÌÇ§ÏßÄ Ï∂©ÎèåÏùÑ Î∞©ÏßÄÌïòÍ∏∞ ÏúÑÌï¥ Î≥ÑÎèÑÎ°ú ÏÑ§ÏπòÌï¥Ïïº Ìï©ÎãàÎã§.
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
if major_version >= 8:
    # ÏÉàÎ°úÏö¥ GPU(Ïòà: Ampere, Hopper GPUs - RTX 30xx, RTX 40xx, A100, H100, L40)Ïóê ÏÇ¨Ïö©ÌïòÏÑ∏Ïöî.
    !pip install --no-deps packaging ninja einops flash-attn xformers trl peft accelerate bitsandbytes
else:
    # Ïò§ÎûòÎêú GPU(Ïòà: V100, Tesla T4, RTX 20xx)Ïóê ÏÇ¨Ïö©ÌïòÏÑ∏Ïöî.
    !pip install --no-deps xformers trl peft accelerate bitsandbytes
pass

In [1]:
from unsloth import FastLanguageModel
max_seq_length=4096
dtype=None
load_in_4bit=True
model,tokenizer=FastLanguageModel.from_pretrained(
    model_name='MLP-KTLim/llama-3-Korean-Bllossom-8B',
    max_seq_length=max_seq_length,
    dtype=dtype,
    load_in_4bit=load_in_4bit
)


ü¶• Unsloth: Will patch your computer to enable 2x faster free finetuning.
==((====))==  Unsloth: Fast Llama patching release 2024.7
   \\   /|    GPU: NVIDIA L4. Max memory: 22.168 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.3.0+cu121. CUDA = 8.9. CUDA Toolkit = 12.1.
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.26.post1. FA2 = True]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [2]:
model=FastLanguageModel.get_peft_model(
    model,
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'o_proj',
        'gate_proj',
        'up_proj',
        'down_proj'
    ],
    bias='none',
    use_gradient_checkpointing='unsloth',
    random_state=123,
    use_rslora=False,
    loftq_config=None
)

Unsloth: Dropout = 0 is supported for fast patching. You are using dropout = 0.05.
Unsloth will patch all other layers, except LoRA matrices, causing a performance hit.
Unsloth 2024.7 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


In [4]:
from datasets import load_dataset
EOS_TOKEN=tokenizer.eos_token
alpaca_prompt="""Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
{}

### Response:
{}"""

def formatting_prompts_func(example):
    instructions=example['instruction']
    outputs=example['output']
    texts=[]
    for instruction,output in zip(instructions,outputs):
        texts.append(alpaca_prompt.format(instruction,output)+EOS_TOKEN)
    return {
        'text':texts
    }

dataset=load_dataset('bigdefence/custom',split='train')
dataset=dataset.map(formatting_prompts_func,batched=True)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

In [7]:
from trl import SFTTrainer
from transformers import TrainingArguments

tokenizer.padding_size='right'
trainer=SFTTrainer(
    model=model,
    tokenizer=tokenizer,
    train_dataset=dataset,
    eval_dataset=dataset,
    dataset_text_field='text',
    max_seq_length=max_seq_length,
    dataset_num_proc=2,
    packing=False,
    args=TrainingArguments(
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4,
        warmup_steps=5,
        num_train_epochs=3,
        max_steps=100,
        do_eval=True,
        evaluation_strategy='steps',
        logging_steps=1,
        learning_rate=2e-4,
        fp16=not torch.cuda.is_bf16_supported(),
        bf16=torch.cuda.is_bf16_supported(),
        optim='adamw_8bit',
        weight_decay=0.01,
        lr_scheduler_type='cosine',
        seed=123,
        output_dir='outputs'
    )
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of ü§ó Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1494: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of ü§ó Transformers. Use `eval_strategy` instead
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1961: FutureWarning: `--push_to_hub_token` is deprecated and will be removed in version 5 of ü§ó Transformers. Use `--hub_token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:269: UserWarning: You passed a `max_seq_length` argument to the SFTTrainer, the value you passed will override the one in the `SFTConfig`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/trl/trainer/sft_trainer.py:283: UserWarning: You pa

Map (num_proc=2):   0%|          | 0/10 [00:00<?, ? examples/s]

max_steps is given, it will override any value given in num_train_epochs


In [8]:
# ÌòÑÏû¨ Î©îÎ™®Î¶¨ ÏÉÅÌÉúÎ•º Î≥¥Ïó¨Ï£ºÎäî ÏΩîÎìú
gpu_stats = torch.cuda.get_device_properties(0)  # GPU ÏÜçÏÑ± Í∞ÄÏ†∏Ïò§Í∏∞
start_gpu_memory = round(
    torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3
)  # ÏãúÏûë Ïãú ÏòàÏïΩÎêú GPU Î©îÎ™®Î¶¨ Í≥ÑÏÇ∞
max_memory = round(
    gpu_stats.total_memory / 1024 / 1024 / 1024, 3
)  # GPUÏùò ÏµúÎåÄ Î©îÎ™®Î¶¨ Í≥ÑÏÇ∞
print(
    f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB."
)  # GPU Ïù¥Î¶ÑÍ≥º ÏµúÎåÄ Î©îÎ™®Î¶¨ Ï∂úÎ†•
print(f"{start_gpu_memory} GB of memory reserved.")  # ÏòàÏïΩÎêú Î©îÎ™®Î¶¨ Ïñë Ï∂úÎ†•

GPU = NVIDIA L4. Max memory = 22.168 GB.
5.633 GB of memory reserved.


In [9]:
trainer_stats=trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 10 | Num Epochs = 100
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 100
 "-____-"     Number of trainable parameters = 41,943,040


In [10]:
# ÏµúÏ¢Ö Î©îÎ™®Î¶¨ Î∞è ÏãúÍ∞Ñ ÌÜµÍ≥ÑÎ•º Î≥¥Ïó¨Ï§çÎãàÎã§.
used_memory = round(
    torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3
)  # ÏÇ¨Ïö©Îêú ÏµúÎåÄ Î©îÎ™®Î¶¨Î•º GB Îã®ÏúÑÎ°ú Í≥ÑÏÇ∞Ìï©ÎãàÎã§.
used_memory_for_lora = round(
    used_memory - start_gpu_memory, 3
)  # LoRAÎ•º ÏúÑÌï¥ ÏÇ¨Ïö©Îêú Î©îÎ™®Î¶¨Î•º GB Îã®ÏúÑÎ°ú Í≥ÑÏÇ∞Ìï©ÎãàÎã§.
used_percentage = round(
    used_memory / max_memory * 100, 3
)  # ÏµúÎåÄ Î©îÎ™®Î¶¨ ÎåÄÎπÑ ÏÇ¨Ïö©Îêú Î©îÎ™®Î¶¨Ïùò ÎπÑÏú®ÏùÑ Í≥ÑÏÇ∞Ìï©ÎãàÎã§.
lora_percentage = round(
    used_memory_for_lora / max_memory * 100, 3
)  # ÏµúÎåÄ Î©îÎ™®Î¶¨ ÎåÄÎπÑ LoRAÎ•º ÏúÑÌï¥ ÏÇ¨Ïö©Îêú Î©îÎ™®Î¶¨Ïùò ÎπÑÏú®ÏùÑ Í≥ÑÏÇ∞Ìï©ÎãàÎã§.
print(
    f"{trainer_stats.metrics['train_runtime']} seconds used for training."
)  # ÌõàÎ†®Ïóê ÏÇ¨Ïö©Îêú ÏãúÍ∞ÑÏùÑ Ï¥à Îã®ÏúÑÎ°ú Ï∂úÎ†•Ìï©ÎãàÎã§.
print(
    # ÌõàÎ†®Ïóê ÏÇ¨Ïö©Îêú ÏãúÍ∞ÑÏùÑ Î∂Ñ Îã®ÏúÑÎ°ú Ï∂úÎ†•Ìï©ÎãàÎã§.
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(
    f"Peak reserved memory = {used_memory} GB."
)  # ÏòàÏïΩÎêú ÏµúÎåÄ Î©îÎ™®Î¶¨Î•º GB Îã®ÏúÑÎ°ú Ï∂úÎ†•Ìï©ÎãàÎã§.
print(
    f"Peak reserved memory for training = {used_memory_for_lora} GB."
)  # ÌõàÎ†®ÏùÑ ÏúÑÌï¥ ÏòàÏïΩÎêú ÏµúÎåÄ Î©îÎ™®Î¶¨Î•º GB Îã®ÏúÑÎ°ú Ï∂úÎ†•Ìï©ÎãàÎã§.
print(
    f"Peak reserved memory % of max memory = {used_percentage} %."
)  # ÏµúÎåÄ Î©îÎ™®Î¶¨ ÎåÄÎπÑ ÏòàÏïΩÎêú Î©îÎ™®Î¶¨Ïùò ÎπÑÏú®ÏùÑ Ï∂úÎ†•Ìï©ÎãàÎã§.
print(
    f"Peak reserved memory for training % of max memory = {lora_percentage} %."
)  # ÏµúÎåÄ Î©îÎ™®Î¶¨ ÎåÄÎπÑ ÌõàÎ†®ÏùÑ ÏúÑÌï¥ ÏòàÏïΩÎêú Î©îÎ™®Î¶¨Ïùò ÎπÑÏú®ÏùÑ Ï∂úÎ†•Ìï©ÎãàÎã§.

485.9731 seconds used for training.
8.1 minutes used for training.
Peak reserved memory = 7.4 GB.
Peak reserved memory for training = 1.767 GB.
Peak reserved memory % of max memory = 33.381 %.
Peak reserved memory for training % of max memory = 7.971 %.


In [14]:
from transformers import StoppingCriteria,StoppingCriteriaList
class stopOnTokens(StoppingCriteria):
    def __init__(self,stop_token_id):
        self.stop_token_id=stop_token_id
    def __call__(self, input_ids, scores, **kwargs):
        return (
            self.stop_token_id in input_ids[0]
        )
stop_token="<|eot_id|>"
stop_token_id=tokenizer.encode(stop_token,add_special_tokens=False)[0]
stopping_criteria=StoppingCriteriaList([stopOnTokens(stop_token_id)])

In [15]:
from transformers import TextStreamer
FastLanguageModel.for_inference(model)
inputs=tokenizer(
    [
        alpaca_prompt.format(
            "Ïπ¥Ïπ¥Ïò§vxÏóê ÎåÄÌï¥ ÏÑ§Î™ÖÌï¥Ï§ò",
            "",
        )
    ],
    return_tensors='pt'
).to('cuda')

text_streamer=TextStreamer(tokenizer)
_=model.generate(
    **inputs,
    streamer=text_streamer,
    max_new_tokens=max_seq_length,
    stopping_criteria=stopping_criteria
)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


<|begin_of_text|>Below is an instruction that describes a task. Write a response that appropriately completes the request.

### Instruction:
Ïπ¥Ïπ¥Ïò§vxÏóê ÎåÄÌï¥ ÏÑ§Î™ÖÌï¥Ï§ò

### Response:
KakaoVXÎäî Ïπ¥Ïπ¥Ïò§ Í≥ÑÏó¥ÏÇ¨Î°ú, Í∞ÄÏÉÅÌòÑÏã§(VR) Î∞è ÌôïÏû•ÌòÑÏã§(XR) Í∏∞Ïà†ÏùÑ ÌôúÏö©Ìïú ÏÑúÎπÑÏä§Î•º Í∞úÎ∞úÌïòÍ≥† Ï†úÍ≥µÌïòÎäî ÌöåÏÇ¨ÏûÖÎãàÎã§. Ï£ºÏöî ÏÇ¨ÏóÖ ÏòÅÏó≠ÏùÄ VR/XR ÏΩòÌÖêÏ∏† Í∞úÎ∞ú, ÌîåÎû´Ìèº Ïö¥ÏòÅ, Í∑∏Î¶¨Í≥† Í¥ÄÎ†® Í∏∞Ïà† Ïó∞Íµ¨ Î∞è Í∞úÎ∞úÏûÖÎãàÎã§.<|eot_id|>


In [16]:
base_model='MLP-KTLim/llama-3-Korean-Bllossom-8B'
huggingface_repo='llama-3-Bllossom-unsloth-finetune'
save_method=(
    'merged_16bit'
)

In [18]:
model.push_to_hub_merged(
    huggingface_repo,
    tokenizer,
    save_method=save_method,
    token='ÌóàÍπÖÌéòÏù¥Ïä§ ÌÜ†ÌÅ∞'
)

Unsloth: Kaggle/Colab has limited disk space. We need to delete the downloaded
model which will save 4-16GB of disk space, allowing you to save on Kaggle/Colab.
Unsloth: Will remove a cached repo with size 16.1G


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 30.96 out of 52.96 RAM for saving.


100%|‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà‚ñà| 32/32 [00:12<00:00,  2.50it/s]


Unsloth: Saving to organization with address bigdefence/llama-3-Bllossom-unsloth-finetune
Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...
Unsloth: Saving to organization with address bigdefence/llama-3-Bllossom-unsloth-finetune
Unsloth: Uploading all files... Please wait...


model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

Upload 4 LFS files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

Done.
Saved merged model to https://huggingface.co/None/llama-3-Bllossom-unsloth-finetune
